In [1]:
# from Bio.Blast import NCBIWWW
from Bio import SeqIO
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastpCommandline
import xml.etree.ElementTree as ET
import csv
import numpy as np
import os
import pandas as pd
import scipy.sparse as sp

## 0. Path settings

In [ ]:
general_path = '/usr/data/cvpr_shared/biology/function/CAFA3/training_data/clustered_70seqid/hhblits_n5_uniclust30_2016_03/BLAST_KNN/BLAST_data_prepare'
# For perpare swissprot data
swissprot_protein_with_go_path=general_path + r"ProteinWithGo.xml"
swissprot_protein_with_go_output_path = general_path + r"SwissprotProteinWithGo.csv"

# For Running BLAST
# path where blastp is installed
blastp = r"/usr/local/ncbi/blast/bin/blastp"
# path of database for blastp, here I download swissprot
swissprot = r"../swissprot/swissprot"

outputfile = general_path+ r"blast_result/"
# taking first 50 results
num_alignments = 50
# SwissprotProteinWithGo = np.load('SwissprotProteinWithGo.npy').item()

## 1. Prepare swissprot data

In [ ]:
def prepare_uniprot_swissprot_data(swissprot_protein_with_go_path,swissprot_protein_with_go_output_path):
    '''
    input: 
    swissprot_protein_with_go_path: swissprot_data, format: .xml,
    each 'entry' contains children 'accession', 'dbReference(Goterms)'
    This file is downloaded with 'curl -O ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.xml.gz'
    and preprocessed with 
    'grep -e '^<entry' 'entry>' 'type="GO"' 'accession' > ProteinWithGo.txt' ||
    'sed 's/\([0-9]*"\)\(>\)/\1\/\2/g' ProteinWithGo.txt '
    
    swissprot_protein_with_go_output_path: path to save output
    
    output: npy file with key = accession, value = Goterms
    '''
    ns = '{http://uniprot.org/uniprot}'
    tree = ET.parse(swissprot_protein_with_go_path)
    root = tree.getroot()
    proteins = root.findall(ns+'entry')
    print('total number of proteins: ',len(proteins))
    Uniprot={}
    for child in root: #get all entries
        dbReferences = child.findall(ns+'dbReference')#get go terms list
        name = child.find(ns+'accession').text#get accession
        Uniprot[name] = []
        for dbReference in dbReferences:
            if dbReference.attrib['type'] == 'GO':
                Uniprot[name].append(dbReference.attrib['id'])
    #Write result to npy file
    # Save
    np.save('SwissprotProteinWithGo.npy', Uniprot) 



In [ ]:
prepare_uniprot_swissprot_data(swissprot_protein_with_go_path,swissprot_protein_with_go_output_path)

In [ ]:
def prepare_uniprot_swissprot_data_parents()
    #Put Go parents term in SwissprotProteinWithGo
    go = np.load("goterm_pairs.npy")
    SwissprotProteinWithGo = np.load('SwissprotProteinWithGo.npy').item()
    for key in SwissprotProteinWithGo:
        if SwissprotProteinWithGo[key]:
            mid = set(SwissprotProteinWithGo[key])
            for goterm in SwissprotProteinWithGo[key]:
                if goterm in go:
                    mid=mid.union(set(go[goterm]))
            SwissprotProteinWithGo[key]=list(mid)
    np.save('SwissprotProteinWithGoParents.npy', SwissprotProteinWithGo)         

## 2. Run BLAST

In [87]:
def blast_knn_output(inputfile,outputfile,swissprot,num_alignments,k):
    """install blast for mac: curl -O ftp://ftp.ncbi.nlm.nih.gov/blast/executables/LATEST/ncbi-blast-2.8.1+.dmg
    download swissprot database: curl -O ftp://ftp.ncbi.nlm.nih.gov/blast/db/swissprot.tar.gz
    input:
    param: inputfile: path to .fasta file of proteins (one sequence in a single file)
    param: outputfile: path to .xml to store blast consquence
    param: swissprot: path to swissprot database (download online )
    param: num_alignments: choose top X blast result for calculating score
    #output: list of (knn_protein_id,score)
    """
    #Command line tool, run blastp on terminal and write result to outputfile
    blastp_cline = NcbiblastpCommandline(blastp,query=inputfile,db=swissprot, evalue=0.001,outfmt=5,num_alignments=num_alignments, out=outputfile)
    blastp_cline()
    #create handle for xml reader
    result_handle = open(outputfile)
    blast_record = NCBIXML.read(result_handle)
    result = []
    #expect value threshold
    E_VALUE_THRESH = 0.04
    i=0
    for alignment in blast_record.alignments:
        i+=1
        #Taking first k proteins with highest score from blast result
        if i>k:
            break
        else:
            for hsp in alignment.hsps:
                #delete alignment with high expect value
                if hsp.expect < E_VALUE_THRESH:
                    #since alignment.title is gi|124356|sp|Q84U76|..., the string Q84U76 is protein_accession and we need
                    #to extract it
                    sp = alignment.title.split('|')[3]
                    sp = sp.split('.')[0]
                    result.append((sp,hsp.score))

        print('finish blast '+ seq.name +' with Swissprot',len(result))
    return result


In [6]:
def calculate_Blastknn_score(blast_knn_result,SwissprotProteinWithGo):
    '''Calculate Blastknn_score for a single protein
    input: 
    blast_knn_result: from blast_knn_output() 
    SwissprotProteinWithGo: from prepare_uniprot_swissprot_data()
    output: list of (Goterm,score)
    '''

    Blastknn_score = {}
    sum_score = 0
    #weight average method to calculate score for each go terms.
    for sp, score in blast_knn_result:
        if SwissprotProteinWithGo.get(sp):
            Goterms = SwissprotProteinWithGo[sp]
            sum_score += score
            for goterm in Goterms:
                if Blastknn_score.get(goterm):
                    Blastknn_score[goterm]+=score
                else:
                    Blastknn_score[goterm]=score
    for goterm in Blastknn_score:
        Blastknn_score[goterm] = Blastknn_score[goterm]/sum_score
    return Blastknn_score

In [92]:
#calculate blast_knn score for all proteins
def calculate_Blastknn_score_forall():
    All_protein_result = {}
    i=0
    directory = r"/usr/data/cvpr_shared/biology/function/CAFA3/training_data/clustered_70seqid/fa"
    #go through all files in the chosen directory
    for filename in os.listdir(directory):
        if filename.endswith(".fasta") :
            #get name clust-xxxxx-xxxxx
            name =  filename.split('.')[0]
            i+=1
            All_protein_result[name] = {}
            path = os.path.join(directory, filename)
            #read protein sequence
            seq = SeqIO.read(path,"fasta")
            All_protein_result[name]['id'] = seq.name
            #blast with swissprot, store blast result in outputpath for futher usage
            outputpath = outputfile+name+".xml"
            result = blast_knn_output(path,outputpath,swissprot,num_alignments,20)
            #calculate score of goterms
            Blastknn_score = calculate_Blastknn_score(result,SwissprotProteinWithGoParents)

            All_protein_result[name]['score'] = Blastknn_score
            print('num '+str(i)+': finish calaulate blast_knn score of '+seq.name+name)
    np.save('Blast_knn_result_20.npy', All_protein_result)

In [90]:
Blast_knn_result_py = np.load('Blast_knn_result.npy').item()
print(Blast_knn_result_py['cluster-09105-00002']['score'])


{'GO:0016021': 1.0, 'GO:0016020': 0.2552013870365431, 'GO:0005750': 0.603360896238997, 'GO:0005739': 0.40430781541744465, 'GO:0005773': 0.20472125900240065, 'GO:0045153': 0.603360896238997, 'GO:0020037': 1.0, 'GO:0046872': 1.0, 'GO:0042776': 0.5533475593491598, 'GO:0006122': 0.603360896238997, 'GO:0005886': 0.2750733528941051, 'GO:0005774': 0.09555881568418245, 'GO:0005743': 0.46399039743931714, 'GO:0070469': 0.3966391037610029, 'GO:0009055': 0.3966391037610029, 'GO:0005634': 0.05048012803414244, 'GO:0045155': 0.05048012803414244, 'GO:0033762': 0.09802614030408109, 'GO:0005829': 0.05034675913576954, 'GO:0005758': 0.05034675913576954, 'GO:0005746': 0.05034675913576954, 'GO:0006119': 0.05034675913576954, 'GO:0022904': 0.033142171245665514}


In [89]:
print(All_protein_result['cluster-25302-00001']['score'])

{'GO:0034464': 0.7426508509541001, 'GO:0060170': 0.9026989857314767, 'GO:0005929': 0.8433900636066701, 'GO:0000242': 0.9026989857314767, 'GO:0034452': 0.7426508509541001, 'GO:0003777': 0.9026989857314767, 'GO:0007098': 0.9026989857314767, 'GO:0060271': 0.641395908543923, 'GO:0034451': 0.3991748323878288, 'GO:0005814': 0.29843561973525873, 'GO:0005813': 0.3991748323878288, 'GO:0036064': 0.3577445418600653, 'GO:0005829': 0.3472580367887227, 'GO:0031514': 0.29843561973525873, 'GO:0005634': 0.395736634003782, 'GO:0032391': 0.29843561973525873, 'GO:0001917': 0.29843561973525873, 'GO:0001750': 0.29843561973525873, 'GO:0043014': 0.29843561973525873, 'GO:0048487': 0.29843561973525873, 'GO:0001103': 0.29843561973525873, 'GO:0030534': 0.29843561973525873, 'GO:0048854': 0.29843561973525873, 'GO:0021987': 0.29843561973525873, 'GO:0016358': 0.29843561973525873, 'GO:0060324': 0.29843561973525873, 'GO:0045444': 0.29843561973525873, 'GO:0060613': 0.29843561973525873, 'GO:0021766': 0.29843561973525873,

## Additional Information
Since BLAST requires each protein having a single .fasta file, if the original dataset is not in such format you can use following code to transfer the dataset into single fasta files.

In [73]:
#generate fasta file of our training data
proteins = pd.read_csv("data/proteins.csv")
coevo_p = np.load("coevo_protein.npy")
for i in range(len(proteins['name'])):
    if proteins['name'][i] in coevo_p:
        path = r"train_sample_coevo/"+proteins['name'][i]+".fasta"
        ofile = open(path, "w")
        ofile.write(">" + proteins['id'][i] + "\n" +proteins['seq'][i] + "\n")
        ofile.close()